In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import wilcoxon, ranksums
import pandas as pd
from enum import Enum
import time
from collections import defaultdict
import math
from opfunu.cec_based import cec2014, cec2020, cec2022
import random

# ====================== Optimization Algorithm Classes ======================

class EVOVariant(Enum):
    ORIGINAL = 1
    ADAPTIVE_STEP = 2
    ENSEMBLE = 3
    QUANTUM_INSPIRED = 4
    HYBRID_DE = 5
    SELF_MODIFIED = 6

class CustomEnergyValleyOptimizer:
    def __init__(self, dimensions, population_size, iterations, bounds, variant):
        self.dimensions = dimensions
        self.population_size = population_size
        self.iterations = iterations
        self.bounds = bounds
        self.variant = variant
        
    def optimize(self, func, runs=1):
        all_results = []
        convergence = []
        
        for run in range(runs):
            population = np.random.uniform(self.bounds[0], self.bounds[1], 
                                         (self.population_size, self.dimensions))
            best_fitness = np.inf
            best_solution = None
            
            for iteration in range(self.iterations):
                fitness = np.array([func(ind) for ind in population])
                
                min_idx = np.argmin(fitness)
                if fitness[min_idx] < best_fitness:
                    best_fitness = fitness[min_idx]
                    best_solution = population[min_idx]
                
                convergence.append((run, iteration, best_fitness))
                
                # Energy Valley Optimization logic
                for i in range(self.population_size):
                    if self.variant == EVOVariant.ORIGINAL:
                        pass  # Implement original variant
                    elif self.variant == EVOVariant.ADAPTIVE_STEP:
                        pass  # Implement adaptive step variant
                    # Implement other variants...
                    
                    population[i] = np.clip(population[i], self.bounds[0], self.bounds[1])
            
            all_results.append(best_fitness)
        
        return {'all_results': np.array(all_results), 'convergence': convergence}

class ParticleSwarmOptimizer:
    def __init__(self, dimensions, population_size, iterations, bounds):
        self.dimensions = dimensions
        self.population_size = population_size
        self.iterations = iterations
        self.bounds = bounds
        
    def optimize(self, func, runs=1):
        all_results = []
        convergence = []
        
        for run in range(runs):
            particles = np.random.uniform(self.bounds[0], self.bounds[1], 
                                       (self.population_size, self.dimensions))
            velocities = np.zeros((self.population_size, self.dimensions))
            personal_best = particles.copy()
            personal_best_fitness = np.array([func(p) for p in particles])
            global_best = particles[np.argmin(personal_best_fitness)]
            global_best_fitness = np.min(personal_best_fitness)
            
            for iteration in range(self.iterations):
                inertia = 0.729
                cognitive = 1.49445
                social = 1.49445
                
                r1 = np.random.rand(self.population_size, self.dimensions)
                r2 = np.random.rand(self.population_size, self.dimensions)
                
                velocities = (inertia * velocities + 
                            cognitive * r1 * (personal_best - particles) + 
                            social * r2 * (global_best - particles))
                particles = particles + velocities
                
                particles = np.clip(particles, self.bounds[0], self.bounds[1])
                
                current_fitness = np.array([func(p) for p in particles])
                
                improved = current_fitness < personal_best_fitness
                personal_best[improved] = particles[improved]
                personal_best_fitness[improved] = current_fitness[improved]
                
                if np.min(current_fitness) < global_best_fitness:
                    global_best = particles[np.argmin(current_fitness)]
                    global_best_fitness = np.min(current_fitness)
                
                convergence.append((run, iteration, global_best_fitness))
            
            all_results.append(global_best_fitness)
        
        return {'all_results': np.array(all_results), 'convergence': convergence}

class GeneticAlgorithm:
    def __init__(self, dimensions, population_size, iterations, bounds, 
                 crossover_rate=0.9, mutation_rate=0.1, tournament_size=3):
        self.dimensions = dimensions
        self.population_size = population_size
        self.iterations = iterations
        self.bounds = bounds
        self.crossover_rate = crossover_rate
        self.mutation_rate = mutation_rate
        self.tournament_size = tournament_size
        
    def optimize(self, func, runs=1):
        all_results = []
        convergence = []
        
        for run in range(runs):
            population = np.random.uniform(self.bounds[0], self.bounds[1], 
                                         (self.population_size, self.dimensions))
            best_fitness = np.inf
            best_solution = None
            
            for iteration in range(self.iterations):
                fitness = np.array([func(ind) for ind in population])
                
                min_idx = np.argmin(fitness)
                if fitness[min_idx] < best_fitness:
                    best_fitness = fitness[min_idx]
                    best_solution = population[min_idx]
                
                convergence.append((run, iteration, best_fitness))
                
                # Selection (Tournament Selection)
                new_population = []
                for _ in range(self.population_size):
                    participants = np.random.choice(range(self.population_size), 
                                                  self.tournament_size, replace=False)
                    winner = participants[np.argmin(fitness[participants])]
                    new_population.append(population[winner])
                
                population = np.array(new_population)
                
                # Crossover (Uniform Crossover)
                for i in range(0, self.population_size-1, 2):
                    if random.random() < self.crossover_rate:
                        mask = np.random.rand(self.dimensions) < 0.5
                        temp = population[i].copy()
                        population[i][mask] = population[i+1][mask]
                        population[i+1][mask] = temp[mask]
                
                # Mutation (Gaussian Mutation)
                for i in range(self.population_size):
                    if random.random() < self.mutation_rate:
                        mutation = np.random.normal(0, 0.1, self.dimensions)
                        population[i] = np.clip(population[i] + mutation, 
                                              self.bounds[0], self.bounds[1])
            
            all_results.append(best_fitness)
        
        return {'all_results': np.array(all_results), 'convergence': convergence}

class DifferentialEvolution:
    def __init__(self, dimensions, population_size, iterations, bounds, F=0.5, CR=0.7):
        self.dimensions = dimensions
        self.population_size = population_size
        self.iterations = iterations
        self.bounds = bounds
        self.F = F
        self.CR = CR
        
    def optimize(self, func, runs=1):
        all_results = []
        convergence = []
        
        for run in range(runs):
            population = np.random.uniform(self.bounds[0], self.bounds[1], 
                                         (self.population_size, self.dimensions))
            best_fitness = np.inf
            best_solution = None
            
            for iteration in range(self.iterations):
                fitness = np.array([func(ind) for ind in population])
                
                min_idx = np.argmin(fitness)
                if fitness[min_idx] < best_fitness:
                    best_fitness = fitness[min_idx]
                    best_solution = population[min_idx]
                
                convergence.append((run, iteration, best_fitness))
                
                for i in range(self.population_size):
                    idxs = [idx for idx in range(self.population_size) if idx != i]
                    a, b, c = population[np.random.choice(idxs, 3, replace=False)]
                    
                    mutant = a + self.F * (b - c)
                    mutant = np.clip(mutant, self.bounds[0], self.bounds[1])
                    
                    cross_points = np.random.rand(self.dimensions) < self.CR
                    if not np.any(cross_points):
                        cross_points[np.random.randint(0, self.dimensions)] = True
                    
                    trial = np.where(cross_points, mutant, population[i])
                    
                    trial_fitness = func(trial)
                    if trial_fitness < fitness[i]:
                        population[i] = trial
                        fitness[i] = trial_fitness
            
            all_results.append(best_fitness)
        
        return {'all_results': np.array(all_results), 'convergence': convergence}

class SimulatedAnnealing:
    def __init__(self, dimensions, iterations, bounds, initial_temp=1000, cooling_rate=0.003):
        self.dimensions = dimensions
        self.iterations = iterations
        self.bounds = bounds
        self.initial_temp = initial_temp
        self.cooling_rate = cooling_rate
        
    def optimize(self, func, runs=1):
        all_results = []
        convergence = []
        
        for run in range(runs):
            current_solution = np.random.uniform(self.bounds[0], self.bounds[1], 
                                               self.dimensions)
            current_fitness = func(current_solution)
            best_solution = current_solution.copy()
            best_fitness = current_fitness
            temperature = self.initial_temp
            
            for iteration in range(self.iterations):
                neighbor = current_solution + np.random.normal(0, 1, self.dimensions)
                neighbor = np.clip(neighbor, self.bounds[0], self.bounds[1])
                neighbor_fitness = func(neighbor)
                
                if neighbor_fitness < current_fitness:
                    current_solution = neighbor
                    current_fitness = neighbor_fitness
                    if neighbor_fitness < best_fitness:
                        best_solution = neighbor
                        best_fitness = neighbor_fitness
                else:
                    prob = np.exp(-(neighbor_fitness - current_fitness) / temperature)
                    if random.random() < prob:
                        current_solution = neighbor
                        current_fitness = neighbor_fitness
                
                convergence.append((run, iteration, best_fitness))
                temperature *= 1 - self.cooling_rate
            
            all_results.append(best_fitness)
        
        return {'all_results': np.array(all_results), 'convergence': convergence}

class GreyWolfOptimizer:
    def __init__(self, dimensions, population_size, iterations, bounds):
        self.dimensions = dimensions
        self.population_size = population_size
        self.iterations = iterations
        self.bounds = bounds
        
    def optimize(self, func, runs=1):
        all_results = []
        convergence = []
        
        for run in range(runs):
            population = np.random.uniform(self.bounds[0], self.bounds[1], 
                                         (self.population_size, self.dimensions))
            fitness = np.array([func(ind) for ind in population])
            
            alpha_pos = population[np.argmin(fitness)]
            alpha_score = np.min(fitness)
            beta_pos = population[np.argsort(fitness)[1]]
            delta_pos = population[np.argsort(fitness)[2]]
            
            best_fitness = alpha_score
            best_solution = alpha_pos.copy()
            
            for iteration in range(self.iterations):
                a = 2 - iteration * (2 / self.iterations)  # a decreases linearly from 2 to 0
                
                for i in range(self.population_size):
                    # Update positions
                    r1 = np.random.rand(self.dimensions)
                    r2 = np.random.rand(self.dimensions)
                    
                    A1 = 2 * a * r1 - a
                    C1 = 2 * r2
                    
                    D_alpha = np.abs(C1 * alpha_pos - population[i])
                    X1 = alpha_pos - A1 * D_alpha
                    
                    r1 = np.random.rand(self.dimensions)
                    r2 = np.random.rand(self.dimensions)
                    
                    A2 = 2 * a * r1 - a
                    C2 = 2 * r2
                    
                    D_beta = np.abs(C2 * beta_pos - population[i])
                    X2 = beta_pos - A2 * D_beta
                    
                    r1 = np.random.rand(self.dimensions)
                    r2 = np.random.rand(self.dimensions)
                    
                    A3 = 2 * a * r1 - a
                    C3 = 2 * r2
                    
                    D_delta = np.abs(C3 * delta_pos - population[i])
                    X3 = delta_pos - A3 * D_delta
                    
                    population[i] = (X1 + X2 + X3) / 3
                    population[i] = np.clip(population[i], self.bounds[0], self.bounds[1])
                
                # Evaluate new solutions
                fitness = np.array([func(ind) for ind in population])
                
                # Update alpha, beta, and delta
                sorted_idx = np.argsort(fitness)
                alpha_pos = population[sorted_idx[0]]
                alpha_score = fitness[sorted_idx[0]]
                beta_pos = population[sorted_idx[1]]
                delta_pos = population[sorted_idx[2]]
                
                if alpha_score < best_fitness:
                    best_fitness = alpha_score
                    best_solution = alpha_pos.copy()
                
                convergence.append((run, iteration, best_fitness))
            
            all_results.append(best_fitness)
        
        return {'all_results': np.array(all_results), 'convergence': convergence}

class AntColonyOptimization:
    def __init__(self, dimensions, population_size, iterations, bounds, 
                 evaporation_rate=0.5, alpha=1, beta=2, q=1):
        self.dimensions = dimensions
        self.population_size = population_size
        self.iterations = iterations
        self.bounds = bounds
        self.evaporation_rate = evaporation_rate
        self.alpha = alpha
        self.beta = beta
        self.q = q
        
    def optimize(self, func, runs=1):
        all_results = []
        convergence = []
        
        for run in range(runs):
            # Initialize pheromone trails
            pheromone = np.ones((self.population_size, self.dimensions))
            
            best_fitness = np.inf
            best_solution = None
            
            for iteration in range(self.iterations):
                # Generate solutions
                solutions = []
                for _ in range(self.population_size):
                    solution = []
                    for d in range(self.dimensions):
                        probabilities = pheromone[:, d]**self.alpha * (1.0 / (np.abs(np.random.rand(self.population_size)) + 1e-10))**self.beta
                        probabilities /= np.sum(probabilities)
                        choice = np.random.choice(range(self.population_size), p=probabilities)
                        val = np.random.uniform(self.bounds[0], self.bounds[1])
                        solution.append(val)
                    solutions.append(solution)
                
                solutions = np.array(solutions)
                
                # Evaluate solutions
                fitness = np.array([func(sol) for sol in solutions])
                
                # Update best solution
                min_idx = np.argmin(fitness)
                if fitness[min_idx] < best_fitness:
                    best_fitness = fitness[min_idx]
                    best_solution = solutions[min_idx]
                
                convergence.append((run, iteration, best_fitness))
                
                # Update pheromones
                pheromone *= (1 - self.evaporation_rate)
                
                for i in range(self.population_size):
                    pheromone[i] += self.q / (fitness[i] + 1e-10)
            
            all_results.append(best_fitness)
        
        return {'all_results': np.array(all_results), 'convergence': convergence}

class HeuristicSearch:
    def __init__(self, dimensions, iterations, bounds):
        self.dimensions = dimensions
        self.iterations = iterations
        self.bounds = bounds
        
    def optimize(self, func, runs=1):
        all_results = []
        convergence = []
        
        for run in range(runs):
            current_solution = np.random.uniform(self.bounds[0], self.bounds[1], 
                                               self.dimensions)
            current_fitness = func(current_solution)
            best_solution = current_solution.copy()
            best_fitness = current_fitness
            
            for iteration in range(self.iterations):
                # Generate neighbor
                neighbor = current_solution + np.random.normal(0, 0.1, self.dimensions)
                neighbor = np.clip(neighbor, self.bounds[0], self.bounds[1])
                neighbor_fitness = func(neighbor)
                
                # Always move to better solutions
                if neighbor_fitness < current_fitness:
                    current_solution = neighbor
                    current_fitness = neighbor_fitness
                    if neighbor_fitness < best_fitness:
                        best_solution = neighbor
                        best_fitness = neighbor_fitness
                
                convergence.append((run, iteration, best_fitness))
            
            all_results.append(best_fitness)
        
        return {'all_results': np.array(all_results), 'convergence': convergence}

# ====================== Benchmark Function Wrappers ======================

class CEC2014Wrapper:
    def __init__(self, func_num, dimensions):
        self.func_num = func_num
        self.dimensions = dimensions
        self.func = getattr(cec2014, f"F{func_num}2014")(ndim=dimensions)
        
    def evaluate(self, x):
        return self.func.evaluate(x)

class CEC2020Wrapper:
    def __init__(self, func_num, dimensions):
        self.func_num = func_num
        self.dimensions = dimensions
        self.func = getattr(cec2020, f"F{func_num}2020")(ndim=dimensions)
        
    def evaluate(self, x):
        return self.func.evaluate(x)

class CEC2022Wrapper:
    def __init__(self, func_num, dimensions):
        self.func_num = func_num
        self.dimensions = dimensions
        self.func = getattr(cec2022, f"F{func_num}2022")(ndim=dimensions)
        
    def evaluate(self, x):
        return self.func.evaluate(x)

# ====================== Engineering Problems ======================

def welded_beam_design(x):
    """Welded Beam Design Problem"""
    x1, x2, x3, x4 = x
    P = 6000
    L = 14
    E = 30e6
    G = 12e6
    tau_max = 13600
    sigma_max = 30000
    delta_max = 0.25
    
    # Derived values
    Pc = (4.013 * E * np.sqrt(x3**2 * x4**6 / 36) / L**2) * (1 - (x3/(2*L))) * np.sqrt(E/(4*G))
    tau1 = P / (np.sqrt(2) * x1 * x2)
    M = P * (L + x2 / 2)
    R = np.sqrt(x2**2 / 4 + ((x1 + x3) / 2)**2)
    J = 2 * (np.sqrt(2) * x1 * x2 * (x2**2 / 12 + ((x1 + x3) / 2))**2)
    tau2 = M * R / J
    tau = np.sqrt(tau1**2 + tau2**2 + tau1 * tau2 * x2 / R)
    sigma = 6 * P * L / (x4 * x3**2)
    delta = 4 * P * L**3 / (E * x3**3 * x4)
    
    # Constraints
    g1 = tau - tau_max
    g2 = sigma - sigma_max
    g3 = x1 - x4
    g4 = 0.10471 * x1**2 + 0.04811 * x3 * x4 * (14 + x2) - 5
    g5 = 0.125 - x1
    g6 = delta - delta_max
    g7 = P - Pc
    
    # Penalty for constraint violation
    penalty = max(0, g1)**2 + max(0, g2)**2 + max(0, g3)**2 + max(0, g4)**2 + max(0, g5)**2 + max(0, g6)**2 + max(0, g7)**2
    
    # Cost function to minimize
    cost = 1.10471 * x1**2 * x2 + 0.04811 * x3 * x4 * (14 + x2)
    return cost + 1e6 * penalty

def pressure_vessel_design(x):
    """Pressure Vessel Design Problem"""
    x1, x2, x3, x4 = x  # x1 and x2 are discrete multiples of 0.0625
    x1 = 0.0625 * np.round(x1 / 0.0625)
    x2 = 0.0625 * np.round(x2 / 0.0625)
    
    # Constraints
    g1 = -x1 + 0.0193 * x3
    g2 = -x2 + 0.00954 * x3
    g3 = -np.pi * x3**2 * x4 - (4/3) * np.pi * x3**3 + 1296000
    g4 = x4 - 240
    
    # Penalty for constraint violation
    penalty = max(0, g1)**2 + max(0, g2)**2 + max(0, g3)**2 + max(0, g4)**2
    
    # Cost function to minimize
    cost = 0.6224 * x1 * x3 * x4 + 1.7781 * x2 * x3**2 + 3.1661 * x1**2 * x4 + 19.84 * x1**2 * x3
    return cost + 1e6 * penalty

def tension_compression_spring(x):
    """Tension/Compression Spring Design Problem"""
    x1, x2, x3 = x
    
    # Constraints
    g1 = 1 - (x2**3 * x3)/(71785 * x1**4)
    g2 = (4*x2**2 - x1*x2)/(12566*(x2*x1**3 - x1**4)) + 1/(5108*x1**2) - 1
    g3 = 1 - (140.45*x1)/(x2**2*x3)
    g4 = (x1 + x2)/1.5 - 1
    
    # Penalty for constraint violation
    penalty = max(0, g1)**2 + max(0, g2)**2 + max(0, g3)**2 + max(0, g4)**2
    
    # Cost function to minimize
    cost = (x3 + 2) * x2 * x1**2
    return cost + 1e6 * penalty

def three_bar_truss_design(x):
    """Three-Bar Truss Design Problem"""
    x1, x2 = x
    
    # Constants
    l = 100
    P = 2
    sigma = 2
    
    # Constraints
    g1 = (np.sqrt(2)*x1 + x2)/(np.sqrt(2)*x1**2 + 2*x1*x2) * P - sigma
    g2 = x2/(np.sqrt(2)*x1**2 + 2*x1*x2) * P - sigma
    g3 = 1/(np.sqrt(2)*x2 + x1) * P - sigma
    
    # Penalty for constraint violation
    penalty = max(0, g1)**2 + max(0, g2)**2 + max(0, g3)**2
    
    # Cost function to minimize
    cost = (2 * np.sqrt(2) * x1 + x2) * l
    return cost + 1e6 * penalty

# ====================== Main Experiment ======================

def run_complete_experiment(population_size=50, iterations=1200, runs=50):
    # Initialize all test functions with appropriate dimensions
    test_functions = {}
    
    # Common dimensions for most functions
    common_dim = 30
    
    # Add CEC2014 functions (1-30) - supports D=30
    for i in range(1, 31):
        test_functions[f"CEC2014_f{i}"] = CEC2014Wrapper(i, common_dim).evaluate
    
    # Add CEC2020 functions (1-10) - supports D=5, 10, 15, 20
    cec2020_dim = 10  # Using 10 as it's a common supported dimension
    for i in range(1, 11):
        test_functions[f"CEC2020_f{i}"] = CEC2020Wrapper(i, cec2020_dim).evaluate
    
    # Add CEC2022 functions (1-12) - supports D=2, 10, 20
    cec2022_dim = 10  # Using 10 as a compromise between complexity and support
    for i in range(1, 13):
        test_functions[f"CEC2022_f{i}"] = CEC2022Wrapper(i, cec2022_dim).evaluate
    
    # Add engineering problems with their specific dimensions
    test_functions["WeldedBeam"] = welded_beam_design  # dim=4
    test_functions["PressureVessel"] = pressure_vessel_design  # dim=4
    test_functions["TensionCompressionSpring"] = tension_compression_spring  # dim=3
    test_functions["ThreeBarTruss"] = three_bar_truss_design  # dim=2
    
    # Initialize all algorithms with appropriate dimensions
    algorithms = {
        'EVO_Original': CustomEnergyValleyOptimizer(common_dim, population_size, iterations, (-100, 100), EVOVariant.ORIGINAL),
        'EVO_AdaptiveStep': CustomEnergyValleyOptimizer(common_dim, population_size, iterations, (-100, 100), EVOVariant.ADAPTIVE_STEP),
        'EVO_Ensemble': CustomEnergyValleyOptimizer(common_dim, population_size, iterations, (-100, 100), EVOVariant.ENSEMBLE),
        'EVO_Quantum': CustomEnergyValleyOptimizer(common_dim, population_size, iterations, (-100, 100), EVOVariant.QUANTUM_INSPIRED),
        'EVO_HybridDE': CustomEnergyValleyOptimizer(common_dim, population_size, iterations, (-100, 100), EVOVariant.HYBRID_DE),
        'EVO_SelfModified': CustomEnergyValleyOptimizer(common_dim, population_size, iterations, (-100, 100), EVOVariant.SELF_MODIFIED),
        'PSO': ParticleSwarmOptimizer(common_dim, population_size, iterations, (-100, 100)),
        'GA': GeneticAlgorithm(common_dim, population_size, iterations, (-100, 100)),
        'DE': DifferentialEvolution(common_dim, population_size, iterations, (-100, 100)),
        'SA': SimulatedAnnealing(common_dim, iterations, (-100, 100)),
        'GWO': GreyWolfOptimizer(common_dim, population_size, iterations, (-100, 100)),
        'ACO': AntColonyOptimization(common_dim, population_size, iterations, (-100, 100)),
        'Heuristic': HeuristicSearch(common_dim, iterations, (-100, 100))
    }
    
    # Run experiments
    results = defaultdict(dict)
    convergence_data = defaultdict(list)
    
    for func_name, func in test_functions.items():
        print(f"\nEvaluating {func_name}...")
        for algo_name, algorithm in algorithms.items():
            print(f"  Running {algo_name}...")
            
            # Adjust dimensions for the current problem
            if func_name.startswith('CEC2020'):
                current_dim = cec2020_dim
            elif func_name.startswith('CEC2022'):
                current_dim = cec2022_dim
            elif func_name in ["WeldedBeam", "PressureVessel"]:
                current_dim = 4
            elif func_name == "TensionCompressionSpring":
                current_dim = 3
            elif func_name == "ThreeBarTruss":
                current_dim = 2
            else:
                current_dim = common_dim
            
            # Set the current dimension for the algorithm
            algorithm.dimensions = current_dim
            
            # Adjust bounds for engineering problems
            if func_name in ["WeldedBeam", "PressureVessel", "TensionCompressionSpring", "ThreeBarTruss"]:
                algorithm.bounds = (0.1, 10) if func_name == "TensionCompressionSpring" else (0.05, 5)
            
            start_time = time.time()
            result = algorithm.optimize(func, runs=runs)
            elapsed = time.time() - start_time
            
            results[func_name][algo_name] = result['all_results']
            
            # Store convergence data for plotting
            for run, iteration, fitness in result['convergence']:
                convergence_data[(func_name, algo_name, run)].append((iteration, fitness))
            
            print(f"    Completed in {elapsed:.2f}s - Best: {np.min(result['all_results']):.4e}")
    
    return results, convergence_data

# ====================== Results Analysis ======================

def perform_wilcoxon_tests(results):
    """Perform Wilcoxon signed-rank tests between EVO variants and other algorithms"""
    wilcoxon_results = []
    
    # Get all algorithm names
    algorithms = list(next(iter(results.values())).keys())
    
    # We'll compare each EVO variant against all other algorithms
    evo_variants = [algo for algo in algorithms if algo.startswith('EVO')]
    other_algos = [algo for algo in algorithms if not algo.startswith('EVO')]
    
    for func_name in results.keys():
        for evo_variant in evo_variants:
            for other_algo in other_algos:
                # Get the results for both algorithms
                evo_results = results[func_name][evo_variant]
                other_results = results[func_name][other_algo]
                
                # Perform Wilcoxon signed-rank test
                stat, p_value = wilcoxon(evo_results, other_results)
                
                # Determine if the difference is significant
                significant = p_value < 0.05
                
                # Determine which is better (lower fitness)
                evo_mean = np.mean(evo_results)
                other_mean = np.mean(other_results)
                better = "EVO" if evo_mean < other_mean else other_algo
                
                wilcoxon_results.append({
                    'Function': func_name,
                    'Comparison': f"{evo_variant} vs {other_algo}",
                    'Statistic': stat,
                    'p-value': p_value,
                    'Significant': significant,
                    'Better': better
                })
    
    return pd.DataFrame(wilcoxon_results)

def generate_statistical_tables(results):
    """Generate tables with mean, std dev, and rankings"""
    # Create comprehensive results table
    results_table = []
    
    for func_name in results.keys():
        for algo_name in results[func_name].keys():
            algo_results = results[func_name][algo_name]
            results_table.append({
                'Function': func_name,
                'Algorithm': algo_name,
                'Mean': np.mean(algo_results),
                'Std Dev': np.std(algo_results),
                'Median': np.median(algo_results),
                'Best': np.min(algo_results),
                'Worst': np.max(algo_results)
            })
    
    results_df = pd.DataFrame(results_table)
    
    # Create rankings table
    rankings = []
    
    for func_name in results.keys():
        # Get all algorithms for this function with their mean performance
        algo_means = [(algo, np.mean(results[func_name][algo])) for algo in results[func_name].keys()]
        
        # Sort by mean performance (ascending)
        algo_means.sort(key=lambda x: x[1])
        
        # Assign ranks
        for rank, (algo, mean) in enumerate(algo_means, 1):
            rankings.append({
                'Function': func_name,
                'Algorithm': algo,
                'Mean': mean,
                'Rank': rank
            })
    
    rankings_df = pd.DataFrame(rankings)
    
    return results_df, rankings_df

def plot_convergence_curves(convergence_data, num_algorithms_to_plot=5):
    """Plot convergence curves for representative functions"""
    # Get unique function names
    func_names = list(set([f for (f, a, r) in convergence_data.keys()]))
    
    # We'll plot the first CEC2014, first CEC2017, and all engineering problems
    plot_funcs = [f for f in func_names if f.startswith('CEC2014_f1') or 
                  f.startswith('CEC2017_f1') or 
                  f in ['WeldedBeam', 'PressureVessel', 'TensionCompressionSpring', 'ThreeBarTruss']]
    
    for func_name in plot_funcs:
        plt.figure(figsize=(12, 8))
        
        # Get all algorithms for this function
        algorithms = list(set([a for (f, a, r) in convergence_data.keys() if f == func_name]))
        
        # Sort algorithms by their final performance (take first run as representative)
        def get_final_perf(algo):
            runs = [r for (f, a, r) in convergence_data.keys() if f == func_name and a == algo]
            if not runs:
                return float('inf')
            # Get the last iteration of the first run
            conv_data = convergence_data[(func_name, algo, runs[0])]
            return conv_data[-1][1] if conv_data else float('inf')
        
        algorithms.sort(key=get_final_perf)
        
        # Plot top N algorithms for clarity
        for algo in algorithms[:num_algorithms_to_plot]:
            # Get all runs for this algorithm and function
            runs = [r for (f, a, r) in convergence_data.keys() if f == func_name and a == algo]
            
            # Prepare median convergence curve
            all_iterations = sorted(set(d[0] for run in runs 
                                  for d in convergence_data[(func_name, algo, run)]))
            median_fitness = []
            
            for it in all_iterations:
                values = [d[1] for run in runs 
                         for d in convergence_data[(func_name, algo, run)] 
                         if d[0] == it]
                if values:  # Only add if we have data
                    median_fitness.append(np.median(values))
            
            # Only plot if we have data
            if median_fitness:
                plt.plot(all_iterations[:len(median_fitness)], median_fitness, 
                        label=f'{algo} (Final: {median_fitness[-1]:.2e})', linewidth=2)
        
        plt.title(f'Convergence Analysis for {func_name}')
        plt.xlabel('Iterations')
        plt.ylabel('Best Fitness (log scale)')
        plt.yscale('log')
        plt.legend()
        plt.grid(True, which="both", ls="--")
        plt.savefig(f'convergence_{func_name}.png', dpi=300, bbox_inches='tight')
        plt.close()

# ====================== Main Execution ======================

if __name__ == "__main__":
    # First install required packages if not already installed
    try:
        import opfunu
    except ImportError:
        print("Installing required packages...")
        import subprocess
        subprocess.run(["pip", "install", "opfunu", "numpy", "scipy", "pandas", "matplotlib"])
    
    print("Starting comprehensive experiment with CEC benchmarks from opfunu...")
    results, convergence_data = run_complete_experiment()
    
    print("\nAnalyzing results...")
    # Generate statistical tables
    results_df, rankings_df = generate_statistical_tables(results)
    
    # Perform Wilcoxon tests
    wilcoxon_df = perform_wilcoxon_tests(results)
    
    # Plot convergence curves
    print("\nGenerating convergence plots...")
    plot_convergence_curves(convergence_data)
    
    # Save results to files
    results_df.to_csv('optimization_results.csv', index=False)
    rankings_df.to_csv('algorithm_rankings.csv', index=False)
    wilcoxon_df.to_csv('wilcoxon_tests.csv', index=False)
    
    # Print summary statistics
    print("\n=== Optimization Results Summary ===")
    print(results_df.groupby('Algorithm').agg({'Mean': ['mean', 'std'], 'Rank': 'mean'}))
    
    print("\n=== Wilcoxon Test Summary ===")
    print("Proportion of significant results where EVO variants outperform:")
    for evo_variant in [a for a in results_df['Algorithm'].unique() if a.startswith('EVO')]:
        evo_tests = wilcoxon_df[wilcoxon_df['Comparison'].str.startswith(evo_variant)]
        better = evo_tests[evo_tests['Better'] == 'EVO']
        print(f"{evo_variant}: {len(better)}/{len(evo_tests)} ({len(better)/len(evo_tests):.1%})")
    
    print("\nExperiment completed successfully! Results saved to CSV files.")

Starting comprehensive experiment with CEC benchmarks from opfunu...

Evaluating CEC2014_f1...
  Running EVO_Original...
    Completed in 344.01s - Best: 1.7953e+09
  Running EVO_AdaptiveStep...
    Completed in 336.17s - Best: 1.1061e+09
  Running EVO_Ensemble...
    Completed in 339.00s - Best: 8.8992e+08
  Running EVO_Quantum...
    Completed in 337.91s - Best: 1.3953e+09
  Running EVO_HybridDE...
    Completed in 335.45s - Best: 2.7615e+09
  Running EVO_SelfModified...
    Completed in 340.48s - Best: 1.9448e+09
  Running PSO...
    Completed in 202.79s - Best: 1.2796e+06
  Running GA...
    Completed in 497.40s - Best: 2.2549e+07
  Running DE...
    Completed in 955.87s - Best: 2.4498e+07
  Running SA...
    Completed in 7.88s - Best: 2.6176e+07
  Running GWO...
    Completed in 627.18s - Best: 1.1626e+08
  Running ACO...
    Completed in 9901.32s - Best: 5.4166e+08
  Running Heuristic...
    Completed in 6.00s - Best: 9.7216e+08

Evaluating CEC2014_f2...
  Running EVO_Original...

KeyboardInterrupt: 

#### 